In [1]:
# Imports & Setup:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import PIL
import PIL.Image
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay

%matplotlib inline

import warnings

warnings.filterwarnings('ignore')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import utils
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

tf.random.set_seed(42)

from google.colab import drive
drive.mount('/content/drive')

import tarfile

fname = "/content/drive/MyDrive/CUB_200_2011.tgz"
fobj = tarfile.open(fname, "r")
fobj.extractall("/content")
fobj.close()

# function to load in and process images
def load_image(path):
    images = Image.open(f'/content/CUB_200_2011/images/{path}') 
    new_image=images.resize((128,128))
    color_image=new_image.convert("RGB")
    arrays1 = np.asarray(color_image)
    floaters= arrays1.astype('float32')
    floaters2=floaters/255.0
    return floaters2

class_map_dir= pd.read_csv('/content/drive/MyDrive/Bird-Classifications/Tables/class_map_dir_output_new_classes_f.csv')

# Create the base model from the pre-trained model MobileNet V2
IMG_SIZE = (128, 128)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False
#base_model.summary()

class_map_dir_train=class_map_dir.loc[class_map_dir['Is Training Image']==1]
list_of_images_train=[load_image(i) for i in class_map_dir_train['Image Name']]
array_of_img_train=np.asarray(list_of_images_train)
list_of_labels_train=class_map_dir_train['Class Id']

class_map_dir_test=class_map_dir.loc[class_map_dir['Is Training Image']==0]
list_of_images_test=[load_image(i) for i in class_map_dir_test['Image Name']]
array_of_img_test=np.asarray(list_of_images_test)
list_of_labels_test=class_map_dir_test['Class Id']

# sanity check length of train datasets
print(len(list_of_images_train))
print(len(list_of_labels_train))

# sanity check length of test datasets
print(len(list_of_images_test))
print(len(list_of_labels_test))

X_train_alt=array_of_img_train
y_train_alt=list_of_labels_train

X_test_alt=array_of_img_test
y_test_alt=list_of_labels_test

X_train_alt = X_train_alt.reshape(X_train_alt.shape[0], 128, 128, 3)
X_test_alt = X_test_alt.reshape(X_test_alt.shape[0], 128, 128, 3)
y_train_alt = utils.to_categorical(y_train_alt,201)
y_test_alt = utils.to_categorical(y_test_alt,201)

Mounted at /content/drive
9420800/9406464 [==============================] - 0s 0us/step
5994
5994
5794
5794


In [2]:
model_t = Sequential([ base_model,
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

model_t.add(Flatten(input_shape = (128, 128, 3)))
model_t.add(Dense(256, activation = 'relu'))
model_t.add(Dense(256, activation = 'relu'))
model_t.add(Dropout(0.1))
model_t.add(Dense(128, activation = 'relu'))
model_t.add(Dense(64, activation = 'relu'))
model_t.add(Dropout(0.2))
model_t.add(Dense(64, activation = 'relu'))
model_t.add(Dropout(0.2))
model_t.add(Dense(48, activation = 'relu'))

model_t.add(Dense(201, activation = 'softmax'))
# Compile model.
model_t.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model_t.trainable = True
early_stop = EarlyStopping(monitor = 'val_accuracy', min_delta=0, patience = 5, verbose = 1, mode = 'auto')
history_t  = model_t.fit(X_train_alt,
                    y_train_alt, 
                    batch_size=256,
                    validation_data=(X_test_alt, y_test_alt),
                    epochs=50,
                    verbose=1,
                    callbacks = [early_stop])

Epoch 1/50
24/24 [==============================] - 18s 265ms/step - loss: 5.3259 - accuracy: 0.0047 - val_loss: 5.2823 - val_accuracy: 0.0117
Epoch 2/50
24/24 [==============================] - 4s 161ms/step - loss: 5.2660 - accuracy: 0.0093 - val_loss: 5.1727 - val_accuracy: 0.0116
Epoch 3/50
24/24 [==============================] - 4s 161ms/step - loss: 5.0881 - accuracy: 0.0157 - val_loss: 4.8798 - val_accuracy: 0.0376
Epoch 4/50
24/24 [==============================] - 4s 164ms/step - loss: 4.7806 - accuracy: 0.0305 - val_loss: 4.5149 - val_accuracy: 0.0431
Epoch 5/50
24/24 [==============================] - 4s 162ms/step - loss: 4.4774 - accuracy: 0.0420 - val_loss: 4.2287 - val_accuracy: 0.0689
Epoch 6/50
24/24 [==============================] - 4s 161ms/step - loss: 4.2152 - accuracy: 0.0559 - val_loss: 4.0005 - val_accuracy: 0.0828
Epoch 7/50
24/24 [==============================] - 5s 195ms/step - loss: 3.9471 - accuracy: 0.0761 - val_loss: 3.7639 - val_accuracy: 0.1034
Epoch

In [3]:
model_t.save('saved_models/class_t.h5')